In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls /content/gdrive

In [ ]:
!curl ipecho.net/plain

35.199.47.114

In [ ]:
%cd /content/gdrive/My\ Drive/Thesis/MovieLens20M

/content/gdrive/My Drive/Thesis/MovieLens20M


In [ ]:
import glob
import os
import pandas as pd
import numpy as np
import shutil

In [ ]:
def load_name(csv_path,id):
    df= pd.read_csv(csv_path)
    ids = df['movieId']
    return (df.loc[ids == id])['title'].values.tolist()
def load_label(csv_path,id):
    df= pd.read_csv(csv_path)
    ids = df['movieId']
    genres_str = (df.loc[ids == id])['genres'].values.tolist()
    if genres_str:
        genres = genres_str[0].split('|')
        return genres
    else:
        return []
def load_all_labels(csv_path):
    download_path = "Data/"
    df= pd.read_csv(csv_path)
    ids =  df['movieId']
    all_labels = []
    for id in ids:
        i = id
        #print(i)
        labels = load_label(csv_path,i)
        all_labels.extend([l for l in labels if l not in all_labels])
        #print(load_name(csv_path,i))
    return all_labels

In [ ]:
def folder_labels_counter(images_path,csv_path,main_genre,save_path):
    directory = os.listdir(images_path)
    folders = [l for l in directory if '.' not in l]
    count =  [0] * len(main_genre)
    for dir in folders:
        current_path = os.path.join(images_path,dir)
        size = len(os.listdir(current_path))
        #print(size)
        if size==0:
            shutil.rmtree(current_path)
            #print("Removed the path because it had 0 elements...", current_path)
        labels = load_label(csv_path,int(dir))
        for l in labels:
             label_id = main_genre.index(l)
             count[label_id] += 1
    for i in range(0,len(count)):
        print(main_genre[i], " has data",count[i])
    return count

In [ ]:
path = "Data"
main_genre = ['Adventure', 'Comedy', 'Fantasy', 'Animation-Children', 'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror', 'Mystery', 'Sci-Fi', 'Documentary', 'Musical']
save_path ="VisualDataset"
csv_path = "movies_filtered_2.csv"
#count = folder_labels_counter(path,csv_path,main_genre,save_path)

In [ ]:
def create_main_genre_folders(path,main_genres):
    for genre in main_genres:
        current_path = os.path.join(path,genre)
        try:
            print("Creating path" , current_path)
            os.mkdir(current_path)
        except FileExistsError:
            print("Already existing directory")

        current_path = os.path.join(path,genre,"train")
        try:
            os.mkdir(current_path)
            print("Creatiing path", current_path)
        except FileExistsError:
            print("Already existing directory")
            
        current_path = os.path.join(path,genre,"validation")
        try:
            os.mkdir(current_path)
            print("Creatiing path", current_path)
        except FileExistsError:
            print("Already existing directory")
        
       

In [ ]:
def fill_folders_seperated(images_path,csv_path,output_path,main_genre,per = 0.3):
    directory = os.listdir(images_path)
    flag = True
    folders = [l for l in directory if '.' not in l]
    count =  [0] * len(main_genre)
    df= pd.read_csv(csv_path)
    data_ids=[]
    names = []
    labels = []
    for dir in folders:
        img_path = os.path.join(images_path,dir) 
        if int(dir) % 3 == 0:
            current_out_path = os.path.join(output_path,"validation")
        else:  
            current_out_path = os.path.join(output_path,"train")
        for img in os.listdir(img_path):
            current_in_path = os.path.join(img_path,img)
            name = dir +"_"+ img
            names.append(name)
            labels.append(load_label(csv_path,int(dir)))
            out = os.path.join(current_out_path, name)
            shutil.copy(current_in_path, out)

    dataframe = {'ID_Frame':names,
        'Genres': labels
        }
    df = pd.DataFrame(dataframe, columns = ['ID_Frame', 'Genres'])
    df.to_csv("current_used_data.csv")
    return data_ids

def fill_folders(images_path,csv_path,output_path,main_genre,reset=False):
    directory = os.listdir(images_path)
    
    folders = [l for l in directory if '.' not in l]
    count = r [0] * len(main_genre)
    df= pd.read_csv(csv_path)

    for dir in folders:
        img_path = os.path.join(images_path,dir) 
        print("Doing file with id ", dir)
        for img in os.listdir(img_path):
            current_in_path = os.path.join(img_path,img)
            name = dir +"_"+ img
            #names.append(name)
            #labels.append(load_label(csv_path,int(dir)))
            out = os.path.join(output_path, name)
            
            if os.path.exists(out) and reset:
                shutil.copy(current_in_path, out)
                print("already exists skipped saving ", out)
            elif  os.path.exists(out):
                print("already exists skipped saving ", out)
                pass
            else:
                shutil.copy(current_in_path, out)
                print("Copying. to the path with name: ", name)
         

In [ ]:
def fill_csv(images_path,csv_path,main_genre,csv_out_name=None,reset=False):
    directory = os.listdir(images_path)
    
    try:
        df= pd.read_csv(csv_path)
        out_df = pd.read_csv(csv_out_name)
    except:  
        names = []
        labels = []
        dataframe = pd.DataFrame(columns = ['ID_Frame', 'Genres'])
        dataframe.to_csv(csv_out_name)
        out_df = pd.read_csv(csv_out_name)
        df=pd.read_csv(csv_path)
    if out_df is not None:
        print("Using existing csv...")
    for img_name in directory:
        id = int(img_name.split('_')[0])
        if not img_name in out_df['ID_Frame']:
            new_row = {'ID_Frame':img_name,'Genres': load_label(csv_path,id)}
            out_df = out_df.append(new_row, ignore_index=True)
            print(out_df)
            out_df.to_csv(csv_out_name,index=False)

In [ ]:

def count_frames(images_path,csv_path,main_genre,reset=False):
    directory = os.listdir(images_path)
    flag = True
    folders = [l for l in directory if '.' not in l]
    count =  [0] * len(main_genre)
    df= pd.read_csv(csv_path)
    
    data_ids=[]
    names = []
    number_frames=[]
    labels = []
    for dir in folders:
        img_path = os.path.join(images_path,dir) 
        print("Doing file with id ", dir)
        print(img_path)
        number_frames.append(len(os.listdir(img_path)))
        labels.append(load_label(csv_path,int(dir)))
        names.append(load_name(csv_path,int(dir)))
        data_ids.append(int(dir))
        

    dataframe = {
        'ID_Frame':data_ids,
        'Name': names,
        '# of Frames': number_frames,
        'Genres': labels,
        
        }
    df = pd.DataFrame(dataframe, columns = ['ID_Frame','Name','# of Frames', 'Genres'])
    
    df.to_csv("statistics.csv",index=False)
    return data_ids

In [ ]:

main_genre = ['Adventure', 'Comedy', 'Fantasy', 'Animation-Children', 'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror', 'Mystery', 'Sci-Fi', 'Documentary', 'Musical']

#count_frames("Data", "movies_filtered_2.csv",main_genre)

In [ ]:
#fill_folders("Data","movies_filtered_2.csv","VisualDataset",main_genre,"current_used_data.csv")

fill_csv("VisualDataset","CsvFiles/movies_filtered_2.csv",main_genre,"CsvFiles/trial.csv")

OSError: ignored

In [ ]:

df2=pd.read_csv("current_used_data.csv")
df2.drop(df2.columns[df2.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
df2.to_csv("current_used_data.csv")


df3=pd.read_csv("current_used_data.csv")
df3=df3[df3['Genres'] != '[]']
df3.to_csv("current_used_data.csv",index=False)